In [6]:
# Lo primero que hacemos es importar todas las librerias necesarias.
#importamos librerias para tratamiento de datos
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:,}'.format
import re
import math as mat

#importamos librerias la visualizacion de datos
import matplotlib.pyplot as plt
import seaborn as sns
import os
%matplotlib inline

In [7]:
# Abrimos el dataset y hacemos un chequeo rápido para saber cantidad de registros y miramos la composición del mismo.
filename = "../../dataset/properati.csv"
data = pd.read_csv(filename)

display(data.sample(n=6))
print(f"(Cantidad de Filas, Cantidad de columnas) -> {data.shape}")

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
45505,45505,sell,house,Banfield,|Argentina|Bs.As. G.B.A. Zona Sur|Lomas de Zam...,Argentina,Bs.As. G.B.A. Zona Sur,"3,436,152.0","-34.7489788,-58.3816503",-34.7489788,...,223.0,"1,030.3030303030305",762.3318385650224,nan,nan,nan,http://www.properati.com.ar/1947c_venta_casa_b...,CODIGO: 031 ubicado en: CAPDEVILLA 780 - Publ...,CHALET EN DOS PLANTAS EN BANFIELD,https://thumbs4.properati.com/3/Lcv_Hvqd2W0sSd...
1393,1393,sell,house,Quilmes,|Argentina|Bs.As. G.B.A. Zona Sur|Quilmes|Quil...,Argentina,Bs.As. G.B.A. Zona Sur,"3,429,651.0","-34.7173032,-58.252284",-34.7173032,...,340.0,838.0952380952382,"1,294.1176470588236",nan,nan,nan,http://www.properati.com.ar/15i6r_venta_casa_q...,"Importante chalet. Hall de ingreso, toilette y...",BARRANCAS DE QUILMES! RESIDENCIAL EXCLUSIVO.,https://thumbs4.properati.com/6/FBDNUfTDSqj4AI...
117081,117081,sell,apartment,Córdoba,|Argentina|Córdoba|Córdoba|,Argentina,Córdoba,"3,860,259.0","-31.4223568,-64.185741",-31.422356800000006,...,nan,nan,nan,nan,1.0,nan,http://www.properati.com.ar/1cd4j_venta_depart...,Departamento en Venta de 1 dorm. en Cordoba,"Depto en Venta Rondeau 74 6ºD interno,N",https://thumbs4.properati.com/8/vtZ1y-klymQ75q...
103419,103419,sell,apartment,Rosario,|Argentina|Santa Fe|Rosario|,Argentina,Santa Fe,"3,838,574.0",NaN,nan,...,94.0,nan,nan,nan,nan,nan,http://www.properati.com.ar/1brxs_venta_depart...,Sergio Villella Inmobiliaria presenta departam...,Departamento - Fisherton,https://thumbs4.properati.com/9/ELueTAgSz9Yc5y...
65105,65105,sell,apartment,Córdoba,|Argentina|Córdoba|Córdoba|,Argentina,Córdoba,"3,860,259.0",NaN,nan,...,nan,nan,nan,nan,nan,"1,500.0",http://www.properati.com.ar/1abs8_venta_depart...,"Departamento de dos dormitorios, con balcon al...",Departamento en General Paz,https://thumbs4.properati.com/1/8SyDORxcISqV1M...
73162,73162,sell,apartment,Boca,|Argentina|Capital Federal|Boca|,Argentina,Capital Federal,"3,436,004.0",NaN,nan,...,66.0,"2,470.5882352941176","3,181.818181818182",nan,nan,1.0,http://www.properati.com.ar/1anda_venta_depart...,Capital Brokers Propiedades | o 15-Estado de ...,Venta 3 ambientes,https://thumbs4.properati.com/9/RIRVqSMLl0GpFH...


(Cantidad de Filas, Cantidad de columnas) -> (121220, 26)


In [ ]:
# Columna price_aprox_usd: revisamos que opciones tiene, o si se repite lo mismo.
print(f'Cantidad de registros con NaN: {data.price_aprox_usd.isnull().sum()}')
print(f'Porcentaje de registros con NaN: {data.price_aprox_usd.isnull().mean()*100}')
print("Contamos cantidad de registros unicos:")
print(data.groupby(['price_aprox_usd']).size().reset_index(name='count'))

In [ ]:
#graficamos el dato por tipo de propiedad con el fin de observar los rangos de precios. En esta grafica podemos observar la variación en la media de los precios que existe de acuerdo al tipo de vivienda.
data.groupby('property_type')['price_aprox_usd'].mean().plot.bar()

In [ ]:
data.price_aprox_usd.describe()

### Detalle de análisis de columna `price_aprox_usd`:

* `price_aprox_usd` -> Esta columna tiene 20.410 datos nulos o su equivalente 16,83%. Consideramos que esta variable es relevante para la proyección, teniendo en cuenta que el valor por metro cuadrado cubierto y total estan calculados en esta moneda, observamos un valor máximo de 46,545,445 alejado significativamente de la media que es de 239,700.
Esta columna podriamos completarla realizando diferentes operaciones:
 
1. Extrayendo este dato de las columnas description y title
2. Diviendo el precio total en usd por el precio del metro cuadrado

### Detalle de valores extremos en la  columna `price_aprox_usd`:

En el catplot elaborado para esta variable comparada por tipo de propiedad podemos observar como en apartamentos se evidencias valores muy por encima de los valores medios, en el caso de las casas se observan algunas propiedades alejadas de la media.

In [ ]:
sns.catplot(data=data, x= 'price_aprox_usd', y="property_type", kind="box", height=5, aspect=5,)


### Regex para rellenar la columna `price_aprox_USD`:

Vamos a extraer datos de las columnas title y description

In [8]:
data[['description', 'title']] = data[['description', 'title']].apply(lambda x: x.astype(str))

#Que pasa si la propiedad vale más de 4 digitos? el valor minimo de propiedad lleno es de 4,666.62 usd, trae tambien el dato dolares y debemos cambiar . por ,

patron = r'U\$D\s?([0-9]*\.?[0-9]*\.?[0-9]*\.?[0-9]*)'

regex = re.compile(patron, flags = re.IGNORECASE)

precio_aprox_usd_titulo=data['title'].replace(np.NAN,"").apply(lambda x: regex.findall(x)[0] if regex.findall(x) else np.NaN)
data['price_aprox_usd'].fillna(precio_aprox_usd_titulo, inplace=True)

precio_aprox_usd_descripcion=data['description'].replace(np.NAN,"").apply(lambda x: regex.findall(x)[0] if regex.findall(x) else np.NaN)
data['price_aprox_usd'].fillna(precio_aprox_usd_descripcion, inplace=True)

print (data.isna().sum()/data.shape[0]*100)
print (f'entradas: {data.shape[0]}')

Unnamed: 0                                    0.0
operation                                     0.0
property_type                                 0.0
place_name                   0.018973766705164163
place_with_parent_names                       0.0
country_name                                  0.0
state_name                                    0.0
geonames_id                    15.440521366111204
lat-lon                         42.52598581092229
lat                             42.52598581092229
lon                             42.52598581092229
price                          16.837155584886982
currency                        16.83798053126547
price_aprox_local_currency     16.837155584886982
price_aprox_usd                14.825936314139582
surface_total_in_m2             32.44349117307375
surface_covered_in_m2          16.422207556508827
price_usd_per_m2               43.394654347467416
price_per_m2                   27.686850354726943
floor                           93.48374855634384


In [34]:
data.price_aprox_usd.isnull().sum()

17972